In [ ]:
# coding: utf-8

import requests
import csv
from bs4 import BeautifulSoup


def get_the_page(url):
    
    r = requests.get(url,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'})
    html_str = r.text
    mypage = BeautifulSoup(html_str,"html.parser")
    
    myitems = mypage.find_all('li',attrs={'class':'sr1-listing-content-cell pois-restaurant-list-cell '})
    
    titles = []
    likes = []
    locations = []
    prices = []
    countries=[]
    styles=[]
    for myitem in myitems:
        
        titles.append(myitem.find('h2').text.strip())
             
        like=myitem.find('span',attrs={'class':'score score-big highlight'})
        if like is not None:       #some item doesn't have like or other features
            likes.append(like.text)
        else:
            like = '0'
            likes.append(like)
    
        locations.append(myitem.find('div',attrs={'class':'icon-info address'}).find('span').text.strip())
    
        prices.append(myitem.find('div',attrs={'class': 'icon-info icon-info-food-price'}).find('span').text)

    countries_materials=mypage.find_all("ul",attrs={'class':'pois-categoryui-list'})
    country_material=[]
    for material in countries_materials:
        country_material.append(material.find_all('li'))

    for li in country_material:
        countries.append(li[0].text.strip())
        if len(li)>1:
            styles.append(li[1].text.strip())
        else:
            styles.append(li[0].text.strip())

    page_content=[]
    for i in range(0,len(titles)):
        page_content.append([titles[i],locations[i],prices[i],countries[i],styles[i],likes[i]])
 
    return page_content
    

In [ ]:
url_page1='https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-21&BookingPeriod=Dinner'

page1 = get_the_page(url_page1)

page_next=[]
for q in range(1,20):
    page_number=str(q+1)
    url_next=url_page1 + "&page=" + page_number
    page_next.extend(get_the_page(url_next))
    
page_all=page1+page_next
#page_all

In [96]:
with open('oepn_rice.csv','w') as f:
    writer = csv.writer(f)
    header = ['Name','Location','Price','Region','Style','Likes']
    writer.writerow(header)
    writer.writerows(page_all)